In [67]:
from datetime import date, timedelta

from Stock import *

s = date(2020,1,1)
e = date(2021,12,20)
tesla = Stock("tsla")
tesla.load_data()
#tesla.add_data_range(s,e,stockpath='pricedata/tsla.csv')
#tesla.save_data()

# adding individual days
#tesla.add_data(s)

done loading $tsla data.


In [68]:
# Analysis
import seaborn as sns
import matplotlib.pyplot as plt
import re

df = tesla.df.dropna()
df['domain'] = df.apply(
    lambda row: re.sub('www3.|www.','',row.link.split('//')[1].split('/')[0]), axis=1)
df1 = df.sample(frac=0.5)
df2 = df.drop(df1.index)
print(len(df))
print(len(df1))
print(len(df2))

4048
2024
2024


d:\dmarq\tools\python37\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [69]:
count = {}
for p, tx in df1[['percent_change','text']].values:
    tx = re.sub('[^a-zA-Z ]',' ',tx)
    for word in tx.split():
        if word.lower() not in count:
            count[word.lower()] = [0,0]
        count[word.lower()][p < 0] += 1

In [70]:
prob = {k:list(np.array(count[k])/(sum(count[k]))) for k in count}

In [71]:
print('--------- positive ---------')
for k in sorted(prob,key=prob.get,reverse=True):
    if prob[k][0] >= 0.70 and sum(count[k]) > 100:
        print(k,prob[k],count[k])
print('--------- negative ---------')
for k in sorted(prob,key=prob.get,reverse=False):
    if prob[k][1] >= 0.70 and sum(count[k]) > 25:
        print(k,prob[k],count[k])

--------- positive ---------
shall [0.958974358974359, 0.041025641025641026] [187, 8]
proposal [0.9279279279279279, 0.07207207207207207] [103, 8]
committee [0.9142857142857143, 0.08571428571428572] [128, 12]
vote [0.8991596638655462, 0.10084033613445378] [107, 12]
corporation [0.8470588235294118, 0.15294117647058825] [144, 26]
notice [0.8320610687022901, 0.16793893129770993] [109, 22]
directors [0.8279569892473119, 0.17204301075268819] [231, 48]
stockholders [0.8044692737430168, 0.19553072625698323] [144, 35]
meeting [0.7859424920127795, 0.21405750798722045] [246, 67]
stockholder [0.7391304347826086, 0.2608695652173913] [102, 36]
board [0.7007168458781362, 0.2992831541218638] [391, 167]
--------- negative ---------
indenture [0.0, 1.0] [0, 62]
polestar [0.014492753623188406, 0.9855072463768116] [1, 68]
supplemental [0.015873015873015872, 0.9841269841269841] [1, 62]
trustee [0.01694915254237288, 0.9830508474576272] [1, 58]
llc [0.03125, 0.96875] [1, 31]
laughter [0.037037037037037035, 0

In [72]:
accuracy = 0
for num in range(len(df2)):
    prediction = [0,0]
    count = [0, 0]
    tx = df2['text'].iloc[num]
    tx = re.sub('[^a-zA-Z ]',' ',tx)
    for word in tx.split():
        if word.lower() in prob:
            prediction[prob[word.lower()][0] < 0.5] += prob[word.lower()][prob[word.lower()][0] < 0.5]
            count[prob[word.lower()][0] < 0.5] += 1
    if count[0] > 0:
        prediction[0] /= count[0]
    if count[1] > 0:
        prediction[1] /= count[1]
    prediction[0] /= sum(prediction)
    prediction[1] =  1 - prediction[0]
    #print(prediction,df2['percent_change'].iloc[num])
    accuracy += int((df2['percent_change'].iloc[num] < 0) == (prediction[0] < 0))
accuracy /= len(df2)
print(accuracy)

d:\dmarq\tools\python37\lib\site-packages\ipykernel_launcher.py:9: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  if __name__ == '__main__':
d:\dmarq\tools\python37\lib\site-packages\ipykernel_launcher.py:10: DeprecationWarning: In future, it will be an error for 'np.bool_' scalars to be interpreted as an index
  # Remove the CWD from sys.path while we load stuff.


0.5059288537549407
